In [ ]:
import os, sys
sys.path.append('../utils')
import pandas as pd
import numpy as np
from queryHelper import prodFetch, adbFetch
from databaseHelper import *
from sheetHelper import *
from datetime import timedelta
import psycopg2
from datetime import datetime
from openpyxl import Workbook

In [ ]:
conn = psycopg2.connect(dbname = "operations_manager_prod", user = "sarthak_sachdev", password = "Sarthak@12345", host = "operation.replica.upgrid.in", port = "5432")

print('PostgreSQL Connection Established')

query1 = '''
select (created_at + INTERVAL '330 minutes')::DATE as date, complainant_id as driverId, issue_id from tickets where deleted_at is null and source in ('Inbound', 'Whatsapp', 'driverApp', 'partnerApp', 'SOS') and complainant_type = '1' and (created_at + INTERVAL '330 minutes')::DATE >= '20250201'
'''

dfTickets = pd.read_sql(query1, conn)

conn.close()

dfTickets

In [ ]:
dfTickets.sort_values(by = ['date'], ascending = True, inplace = True)
dfTickets = dfTickets.reset_index(drop=True)

dfTickets

In [ ]:
conn = psycopg2.connect(dbname = "operations_manager_prod", user = "sarthak_sachdev", password = "Sarthak@12345", host = "operation.replica.upgrid.in", port = "5432")

print('PostgreSQL Connection Established')

query2 = '''
select id as issue_id, name as issue_name, issue_category_id from issues where deleted_at is null
'''

dfIssues = pd.read_sql(query2, conn)

conn.close()

print(dfIssues)

conn = psycopg2.connect(dbname = "operations_manager_prod", user = "sarthak_sachdev", password = "Sarthak@12345", host = "operation.replica.upgrid.in", port = "5432")

print('PostgreSQL Connection Established')

query3 = '''
select id as issue_category_id, name as category_name from issue_categories where deleted_at is null
'''

dfCategories = pd.read_sql(query3, conn)

conn.close()

print(dfCategories)

In [ ]:
dfTickets = dfTickets.merge(dfIssues, how='left', on = 'issue_id')

dfTickets = dfTickets.merge(dfCategories, how = 'left', on = 'issue_category_id')

dfTickets

In [ ]:
dfTickets.drop(columns = ['issue_category_id', 'issue_id'], inplace= True)

dfTickets.dropna(inplace=True)

dfTickets

In [ ]:
dfDrivers = prodFetch("""select id, zoneId as driverZone from drivers """)

dfTickets = dfTickets.merge(dfDrivers, left_on = 'driverid', right_on = 'id', how = 'left').drop(columns = ['id'])

dfTickets

In [ ]:
dfTxns = prodFetch("""select date_format(date, '%Y-%m') month, date, driverId, 1 as opsFlag, count(*) as txns from transactions where deletedAt is null and clientId in ('BS00') and date >= '20250201' group by month, date, driverId""")

dfTxns

In [ ]:
dfTxns

In [ ]:
dfTickets.rename(columns={'driverid': 'driverId'}, inplace=True)
dfTickets

In [ ]:
dfTickets['date'] = pd.to_datetime(dfTickets['date'])
dfTxns['date'] = pd.to_datetime(dfTxns['date'])

In [ ]:
dfTickets.info()

In [ ]:
dfTxns.info()

In [ ]:
dfTicketsFinal = dfTickets.merge(dfTxns[['driverId', 'date', 'opsFlag', 'txns']], how = 'left', on=['driverId', 'date']).fillna(0)

dfTicketsFinal

In [ ]:
dfTicketsFinal['month'] = dfTicketsFinal['date'].dt.strftime('%Y-%m')
dfTicketsFinal

In [ ]:
dfTicketsFinal.describe()

In [ ]:
dfDrivers = adbFetch("""select date_format(date, '%Y-%m') month, zoneId, count(distinct driverId) as totalDrivers from dailyDriversHistories force index(dailyDriversHistories_date_driverId_unique) where date >= '20250201' and deletedAt is NULL and driverId like 'D%' and isBaasDriver != 1 and clientId = 'BS00' and status in ('active', 'inactive') group by month, zoneId""")

dfDrivers

In [ ]:
dfdsh = prodFetch("""select date_format(date, '%Y-%m') month, date, driverId, status from driverStatusHistories force index(driverId) where driverId like 'D%' and date >= '20250201' and deletedAt is null and status in ('active') and isBaasDriver != 1 and isDeliveryDriver != 1 """)

dfdsh

In [ ]:
dfDriver = prodFetch("""select id as driverId, zoneId from drivers""")

dfDshFinal = dfdsh.merge(dfDriver, how='left', on=['driverId'])

dfDshFinal

In [ ]:
dailyActiveDriversCount = dfDshFinal[dfDshFinal['status'] == 'active'].groupby(['date', 'zoneId']).agg(
    totalActiveDrivers=('driverId', 'nunique')
).reset_index()

dailyActiveDriversCount['month'] = pd.to_datetime(dailyActiveDriversCount['date']).dt.strftime('%Y-%m')

monthlyActiveDriversCount = dailyActiveDriversCount.groupby(['month', 'zoneId']).agg(
    totalActiveDrivers=('totalActiveDrivers', 'sum')
).reset_index()

In [ ]:
monthlyActiveDriversCount

In [ ]:
april_data = dfTicketsFinal[dfTicketsFinal['month'] == '2025-04']

zone_ticket_counts = april_data.groupby('zoneId').size()

top_10_zones = zone_ticket_counts.sort_values(ascending=False).head(10)

ops_driver_counts = april_data[april_data['opsFlag'] == 1].groupby('zoneId')['driverId'].nunique()

In [ ]:
result_top_10_zones = pd.DataFrame({
    'ticketCount': zone_ticket_counts[top_10_zones.index],
    'driverCount': dfDrivers['totalDrivers'],
    'activeDriverCount': monthlyActiveDriversCount['totalActiveDrivers'],
    'opsDriverCount': ops_driver_counts,
    'UDRT': april_data.groupby('zoneId')['2025-04 UDRT'].sum(),
    'totalTxns': april_data.groupby('zoneId')['2025-04 totalTxns'].sum()
})

result_top_10_zones['ops/activePercentage'] = 
total_ticket_count = april_data['2025-04 tickets'].sum()
result_top_10_zones['ticketCountPercentage'] = (result_top_10_zones['ticketCount'] / total_ticket_count) * 100

result_top_10_zones

In [ ]:
march_data = dfTicketsFinal[dfTicketsFinal['month'] == '2025-03']
april_data = dfTicketsFinal[dfTicketsFinal['month'] == '2025-04']

march_category_counts = march_data.groupby(['zoneId', 'category_name'])['2025-03 tickets'].sum()
april_category_counts = april_data.groupby(['zoneId', 'category_name'])['2025-04 tickets'].sum()

category_increase = april_category_counts - march_category_counts

top_10_zones_categories = category_increase[category_increase.index.get_level_values('zoneId').isin(top_10_zones.index)]

top_10_categories = top_10_zones_categories.groupby('category_name').sum().sort_values(ascending=False).head(10)

top_categories_data = april_data[april_data['category_name'].isin(top_10_categories.index)]
category_summary = top_categories_data.groupby(['zoneId', 'category_name']).agg({
    '2025-04 tickets': 'sum',
    '2025-04 UDRT': 'sum',
    '2025-04 UDT': 'sum',
    '2025-04 totalTxns': 'sum'
})

category_summary

In [ ]:
april_data = dfTicketsFinal[dfTicketsFinal['month'] == '2025-04']

daywise_data = april_data.groupby(['date', 'driverZone', 'category_name'])['2025-04 tickets'].sum().reset_index()

daywise_stats = daywise_data.groupby(['driverZone', 'category_name'])['2025-04 tickets'].agg(['mean', 'std'])

daywise_data['score'] = daywise_data.apply(
    lambda row: abs(row['2025-04 tickets'] - daywise_stats.loc[(row['driverZone'], row['category_name']), 'mean']) / 
                daywise_stats.loc[(row['driverZone'], row['category_name']), 'std'], axis=1)

sorted_daywise_data = daywise_data.sort_values(by='score', ascending=False)

sorted_daywise_data

In [ ]:
last_3_months = dfTicketsFinal[dfTicketsFinal['month'].isin(['2025-02', '2025-03', '2025-04'])]

issue_counts = last_3_months.groupby(['category_name', 'issue_name', 'month'])['tickets'].sum().reset_index()

issue_counts['rank'] = issue_counts.groupby(['category_name', 'issue_name'])['tickets'].rank(ascending=False)

issue_rank_sum = issue_counts.groupby(['category_name', 'issue_name'])['rank'].sum()

sorted_issues = issue_rank_sum.sort_values().reset_index()

sorted_issues

# ------------------------------------

In [ ]:
category_ticket_counts = dfTicketsFinal.groupby(['month', 'category_name']).size().reset_index(name='ticketCount')

category_ticket_counts = category_ticket_counts[category_ticket_counts['month'] != '2025-02']

category_ticket_counts

In [ ]:
category_ticket_counts_pivot = category_ticket_counts.pivot_table(
    index='category_name', 
    columns='month', 
    values='ticketCount', 
    aggfunc='sum', 
    fill_value=0
)

category_ticket_counts_pivot['ticketCountDifference'] = category_ticket_counts_pivot['2025-04'] - category_ticket_counts_pivot['2025-03']

category_ticket_counts_sorted = category_ticket_counts_pivot.sort_values(by='ticketCountDifference', ascending=False)

category_ticket_counts_sorted

In [ ]:
category_ticket_counts_sorted = category_ticket_counts_sorted.reset_index()

category_ticket_counts_sorted

In [ ]:
write("https://docs.google.com/spreadsheets/d/1V6K9cTJplBdFxUIX414xqDN1TetXZwm2GNjrvA7JSNI/edit?gid=0#gid=0", "march vs april ticket count", category_ticket_counts_sorted)

In [ ]:
top_10_categories = category_ticket_counts_sorted.head(10)['category_name']

df_filtered_categories = dfTicketsFinal[dfTicketsFinal['category_name'].isin(top_10_categories)]

result_list = []

for category in top_10_categories:
    category_data = df_filtered_categories[df_filtered_categories['category_name'] == category]

    zone_ticket_counts = category_data.groupby('driverZone').size().sort_values(ascending=False).head(10)

    top_10_zones = zone_ticket_counts.index

    result = pd.DataFrame({
        'ticketCount': zone_ticket_counts,
        'ticketCountPercentage': (zone_ticket_counts / category_data.shape[0]) * 100,
        'opsDriverCount': category_data[category_data['opsFlag'] == 1].groupby('driverZone')['driverId'].nunique().loc[top_10_zones],
        'UDRT': category_data.groupby('driverZone')['driverId'].nunique().loc[top_10_zones],
        'totalTxns': category_data.groupby('driverZone')['txns'].sum().loc[top_10_zones]
    })

    result['category_name'] = category
    result_list.append(result)

final_result = pd.concat(result_list)

final_result.reset_index(inplace=True)
final_result

In [ ]:
final_result.sort_values(['driverZone','ticketCount'], ascending=False, inplace=True)
final_result.reset_index(drop=True, inplace=True)
final_result

In [ ]:
write("https://docs.google.com/spreadsheets/d/1V6K9cTJplBdFxUIX414xqDN1TetXZwm2GNjrvA7JSNI/edit?gid=1436888172#gid=1436888172", "top 10 category's top 10 zones", final_result)

In [ ]:
top_10_categories = category_ticket_counts_sorted.head(10)['category_name']

april_data = dfTicketsFinal[(dfTicketsFinal['month'] == '2025-04') & (dfTicketsFinal['category_name'].isin(top_10_categories))]

april_ticket_counts = april_data.groupby(['category_name', 'issue_name']).size().reset_index(name='ticketCount_April')

march_data = dfTicketsFinal[(dfTicketsFinal['month'] == '2025-03') & (dfTicketsFinal['category_name'].isin(top_10_categories))]

march_ticket_counts = march_data.groupby(['category_name', 'issue_name']).size().reset_index(name='ticketCount_March')

merged_data = pd.merge(april_ticket_counts, march_ticket_counts, on=['category_name', 'issue_name'], how='left')

merged_data['ticketCountDifference'] = merged_data['ticketCount_April'] - merged_data['ticketCount_March']

increased_issues = merged_data[merged_data['ticketCountDifference'] > 0]

sorted_increased_issues = increased_issues.sort_values(by='ticketCountDifference', ascending=False)

sorted_increased_issues.reset_index(drop = True, inplace=True)
sorted_increased_issues

In [ ]:
write("https://docs.google.com/spreadsheets/d/1V6K9cTJplBdFxUIX414xqDN1TetXZwm2GNjrvA7JSNI/edit?gid=1436888172#gid=1436888172", "issues where tickets increased since march", sorted_increased_issues)

# month-wise category-wise

In [ ]:
monthly_category_summary = dfTicketsFinal.groupby(['month', 'category_name']).agg(
    tickets=('driverId', 'count'),
    UDRT=('driverId', 'nunique'),
    UDT=('opsFlag', lambda x: dfTicketsFinal.loc[x.index][dfTicketsFinal['opsFlag'] == 1]['driverId'].nunique()),
    totalTxns=('txns', 'sum')
).reset_index()

monthly_category_summary

# monthwise categorywise for top 5 issues

In [ ]:
top_issues = (dfTicketsFinal.groupby(['month', 'category_name', 'issue_name'])
              .size()
              .reset_index(name='ticket_count')
              .sort_values(['month', 'category_name', 'ticket_count'], ascending=[True, True, False]))

top_issues = top_issues.groupby(['month', 'category_name']).head(5)

dfTopIssues = dfTicketsFinal.merge(top_issues[['month', 'category_name', 'issue_name']], on=['month', 'category_name', 'issue_name'])

monthly_issue_summary = dfTopIssues.groupby(['month', 'category_name', 'issue_name']).agg(
    tickets=('driverId', 'count'),
    UDRT=('driverId', 'nunique'),
    UDT=('opsFlag', lambda x: dfTopIssues.loc[x.index][dfTopIssues['opsFlag'] == 1]['driverId'].nunique()),
    totalTxns=('txns', 'sum'),
    totalDrivers=('driverId', 'nunique')
).reset_index()

monthly_issue_summary

# daywise tickets count

In [ ]:
daily_ticket_count = dfTicketsFinal.groupby('date').agg(
    tickets=('driverId', 'count')
).reset_index()

daily_ticket_count

# zonewise monthly tickets

In [ ]:
zonewise_monthly_count = dfTicketsFinal.groupby(['driverZone', 'month']).agg(
    tickets=('driverId', 'count')
).reset_index()

zonewise_monthly_count

# zonewise monthly categoryWise

In [ ]:
zonewise_monthly_category_count = dfTicketsFinal.groupby(['driverZone', 'month', 'category_name']).agg(
    tickets=('driverId', 'count')
).reset_index()

zonewise_monthly_count

# zonewise monthly categoryWise issuewise

In [ ]:
zonewise_monthly_issue_count = dfTicketsFinal.groupby(['driverZone', 'month', 'issue_name']).agg(
    tickets=('driverId', 'count')
).reset_index()

zonewise_monthly_issue_count

# zonewise datewise

In [ ]:
zonewise_datewise_count = dfTicketsFinal.groupby(['driverZone', 'date']).agg(
    tickets=('driverId', 'count')
).reset_index()

zonewise_datewise_count

# graphs

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

sns.set(style="whitegrid")  # Clean seaborn theme

# 1. Monthly Category-wise Tickets
def plot_monthly_category_summary(df):
    plt.figure(figsize=(14, 6))
    sns.barplot(data=df, x='month', y='tickets', hue='category_name')
    plt.title('Monthly Category-wise Tickets')
    plt.xticks(rotation=45)
    plt.legend(title='Category', bbox_to_anchor=(1.05, 1), loc='upper left')
    plt.tight_layout()
    plt.show()

# 2. Monthly Issue-wise for Top 5 Issues
def plot_monthly_issue_summary(df):
    g = sns.FacetGrid(df, col='category_name', col_wrap=3, height=5, sharey=False)
    g.map_dataframe(sns.barplot, x='month', y='tickets', hue='issue_name', dodge=True)
    g.set_titles('{col_name}')
    g.add_legend(title='Issue Name')
    plt.subplots_adjust(top=0.9)
    g.fig.suptitle('Monthly Issue-wise Tickets (Top 5 Issues per Category)')
    plt.show()

# 3. Day-wise Tickets Created
def plot_daily_ticket_count(df):
    plt.figure(figsize=(16, 6))
    sns.lineplot(data=df, x='date', y='tickets', marker='o')
    plt.title('Daily Ticket Creation Trend')
    plt.xticks(rotation=45)
    plt.tight_layout()
    plt.show()

# 4. Zone-wise Monthly Tickets
def plot_zonewise_monthly_count(df):
    plt.figure(figsize=(14, 6))
    sns.barplot(data=df, x='month', y='tickets', hue='driverZone')
    plt.title('Zone-wise Monthly Ticket Counts')
    plt.xticks(rotation=45)
    plt.legend(title='Zone', bbox_to_anchor=(1.05, 1), loc='upper left')
    plt.tight_layout()
    plt.show()

# 5. Zone-wise Monthly Category-wise Tickets
def plot_zonewise_monthly_category_count(df):
    g = sns.FacetGrid(df, col='driverZone', col_wrap=3, height=5, sharey=False)
    g.map_dataframe(sns.barplot, x='month', y='tickets', hue='category_name')
    g.set_titles('Zone: {col_name}')
    g.add_legend(title='Category')
    plt.subplots_adjust(top=0.9)
    g.fig.suptitle('Zone-wise Monthly Category Ticket Counts')
    plt.show()

# 6. Zone-wise Monthly Issue-wise Tickets
def plot_zonewise_monthly_issue_count(df):
    g = sns.FacetGrid(df, col='driverZone', col_wrap=3, height=5, sharey=False)
    g.map_dataframe(sns.barplot, x='month', y='tickets', hue='issue_name')
    g.set_titles('Zone: {col_name}')
    g.add_legend(title='Issue Name')
    plt.subplots_adjust(top=0.9)
    g.fig.suptitle('Zone-wise Monthly Issue Ticket Counts')
    plt.show()

# 7. Zone-wise Date-wise Tickets
def plot_zonewise_datewise_count(df):
    plt.figure(figsize=(16, 6))
    sns.lineplot(data=df, x='date', y='tickets', hue='driverZone', marker='o')
    plt.title('Zone-wise Daily Ticket Trend')
    plt.xticks(rotation=45)
    plt.tight_layout()
    plt.show()


In [ ]:
# plot_monthly_category_summary(monthly_category_summary)
# plot_monthly_issue_summary(monthly_issue_summary)
# plot_daily_ticket_count(daily_ticket_count)
# plot_zonewise_monthly_count(zonewise_monthly_count)
# plot_zonewise_monthly_category_count(zonewise_monthly_category_count)
# plot_zonewise_monthly_issue_count(zonewise_monthly_issue_count)
# plot_zonewise_datewise_count(zonewise_datewise_count)

In [ ]:
top_issues_zone_category = (dfTicketsFinal.groupby(['month', 'driverZone', 'category_name', 'issue_name'])
                            .size()
                            .reset_index(name='ticket_count')
                            .sort_values(['month', 'driverZone', 'category_name', 'ticket_count'], ascending=[True, True, True, False]))

top_issues_zone_category = top_issues_zone_category.groupby(['month', 'driverZone', 'category_name']).head(5)

dfTopIssuesZC = dfTicketsFinal.merge(
    top_issues_zone_category[['month', 'driverZone', 'category_name', 'issue_name']],
    on=['month', 'driverZone', 'category_name', 'issue_name']
)

agg_top_issues_zc = dfTopIssuesZC.groupby(['driverZone', 'category_name', 'issue_name', 'month']).agg(
    tickets=('driverId', 'count'),
    UDRT=('driverId', 'nunique'),
    UDT=('opsFlag', lambda x: dfTopIssuesZC.loc[x.index][dfTopIssuesZC['opsFlag'] == 1]['driverId'].nunique()),
    totalTxns=('txns', 'sum')
).reset_index()

pivot_table = agg_top_issues_zc.pivot_table(
    index=['driverZone', 'category_name', 'issue_name'],
    columns='month',
    values=['tickets', 'UDRT', 'UDT', 'totalTxns'],
    fill_value=0
)

pivot_table.columns = [f'{month} {metric}' for metric, month in pivot_table.columns]
pivot_table = pivot_table.reset_index()

pivot_table

In [ ]:
ticket_cols = [col for col in pivot_table.columns if 'tickets' in col]

zone_ticket_counts = pivot_table.groupby('driverZone')[ticket_cols].sum()

zone_ticket_counts['Total_Tickets'] = zone_ticket_counts.sum(axis=1)

top5_zones = zone_ticket_counts.sort_values('Total_Tickets', ascending=False).head(5)

top5_zones_list = top5_zones.index.tolist()
filtered_pivot_table = pivot_table[pivot_table['driverZone'].isin(top5_zones_list)]

filtered_pivot_table

In [ ]:
ticket_cols = [col for col in filtered_pivot_table.columns if 'tickets' in col]

category_ticket_counts = filtered_pivot_table.groupby('category_name')[ticket_cols].sum()

category_ticket_counts['Total_Tickets'] = category_ticket_counts.sum(axis=1)

top10_categories = category_ticket_counts.sort_values('Total_Tickets', ascending=False).head(10)

top10_categories_list = top10_categories.index.tolist()
further_filtered_table = filtered_pivot_table[filtered_pivot_table['category_name'].isin(top10_categories_list)]

further_filtered_table

In [ ]:
ticket_cols = [col for col in further_filtered_table.columns if 'tickets' in col]

further_filtered_table['Total_Tickets'] = further_filtered_table[ticket_cols].sum(axis=1)

sorted_table = further_filtered_table.sort_values(by=['driverZone', 'category_name', 'Total_Tickets'], ascending=[True, True, False])

sorted_table.reset_index(drop = True, inplace=True)
sorted_table